In [1]:

#Bibliotecas para poder trabajar con Spark
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.2//spark-3.2.2-bin-hadoop3.2.tgz
!tar xf spark-3.2.2-bin-hadoop3.2.tgz
#Configuración de Spark con Python
!pip install -q findspark
!pip install pyspark

from IPython.display import Image

#Estableciendo variable de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.2-bin-hadoop3.2"

#Buscando e inicializando la instalación de Spark
import findspark
findspark.init()
findspark.find()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,038 kB]
Hit:1

'/content/spark-3.2.2-bin-hadoop3.2'

In [2]:
from google.colab import drive

drive.mount("/content/gdrive")
%cd "/content/gdrive/Shareddrives/Reto AI SEPTIMO"

Mounted at /content/gdrive
/content/gdrive/Shareddrives/Reto AI SEPTIMO


In [3]:
#Verificar la funcionalidad de Pyspark 
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType,FloatType 
spark_session = SparkSession.builder.appName('Entregable_Mod1').config("spark.driver.memory", "24g").getOrCreate()
spark_session


Considerando una versión del set de datos empleado en el reto, en el cual ya se pasó por un proceso de label encoding y PCA. Se lee el archivo csv y se convierte en un dataframe de spark:

In [4]:
df_spark = spark_session.read.csv('telcom_numeric.csv')
df_spark

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string, _c17: string, _c18: string, _c19: string, _c20: string, _c21: string, _c22: string, _c23: string, _c24: string, _c25: string, _c26: string, _c27: string]

In [5]:
df_spark.show()

+----+------+-----------+------------+------------------+------------------+--------------------+------------------+-------------+--------------+-----------+------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-----------------+-------------+-------------+----------+------------+--------------------+---------------+------+--------------------+-----------------+
| _c0|   _c1|        _c2|         _c3|               _c4|               _c5|                 _c6|               _c7|          _c8|           _c9|       _c10|              _c11|                _c12|                _c13|                _c14|                _c15|              _c16|                _c17|             _c18|         _c19|         _c20|      _c21|        _c22|                _c23|           _c24|  _c25|                _c26|             _c27|
+----+------+-----------+------------+------------------+------------------+----------------

Se obtienen los nombres de las columnas y sus tipos de dato

In [6]:
#La opción option('header','true')
df_spark_col  = spark_session.read.option('header', 'true').csv('telcom_numeric.csv')
df_spark_col

DataFrame[_c0: string, TARGET: string, YEAR_JOINED: string, CURRENT_YEAR: string, BILL_AMOUNT: string, PAID_AMOUNT: string, PAYMENT_TRANSACTIONS: string, PARTY_REV: string, PREPAID_LINES: string, POSTPAID_LINES: string, OTHER_LINES: string, LINE_REV: string, MOUS_TO_LOCAL_MOBILES: string, MOUS_FROM_LOCAL_MOBILES: string, MOUS_TO_LOCAL_LANDLINES: string, MOUS_FROM_LOCAL_LANDLINES: string, MOUS_TO_INT_NUMBER: string, MOUS_FROM_INT_NUMBER: string, DATA_IN_BNDL: string, DATA_OUT_BNDL: string, DATA_USG_PAYG: string, COMPLAINTS: string, Years_stayed: string, PTY_PROFILE_SUB_TYPE: string, PARTY_GENDER_CD: string, STATUS: string, SOCIO_ECONOMIC_SEGMENT: string, PARTY_NATIONALITY: string]

Se dropean todos los posibles NaNs que se presenten en el set de datos

In [7]:
df_spark_col = df_spark_col.na.drop(how = "any")
df_spark_col

DataFrame[_c0: string, TARGET: string, YEAR_JOINED: string, CURRENT_YEAR: string, BILL_AMOUNT: string, PAID_AMOUNT: string, PAYMENT_TRANSACTIONS: string, PARTY_REV: string, PREPAID_LINES: string, POSTPAID_LINES: string, OTHER_LINES: string, LINE_REV: string, MOUS_TO_LOCAL_MOBILES: string, MOUS_FROM_LOCAL_MOBILES: string, MOUS_TO_LOCAL_LANDLINES: string, MOUS_FROM_LOCAL_LANDLINES: string, MOUS_TO_INT_NUMBER: string, MOUS_FROM_INT_NUMBER: string, DATA_IN_BNDL: string, DATA_OUT_BNDL: string, DATA_USG_PAYG: string, COMPLAINTS: string, Years_stayed: string, PTY_PROFILE_SUB_TYPE: string, PARTY_GENDER_CD: string, STATUS: string, SOCIO_ECONOMIC_SEGMENT: string, PARTY_NATIONALITY: string]

In [8]:
df_spark_col.show()

+---+------+-----------+------------+------------------+------------------+--------------------+------------------+-------------+--------------+-----------+------------------+---------------------+-----------------------+-----------------------+-------------------------+------------------+--------------------+-----------------+-------------+-------------+----------+------------+--------------------+---------------+------+----------------------+-----------------+
|_c0|TARGET|YEAR_JOINED|CURRENT_YEAR|       BILL_AMOUNT|       PAID_AMOUNT|PAYMENT_TRANSACTIONS|         PARTY_REV|PREPAID_LINES|POSTPAID_LINES|OTHER_LINES|          LINE_REV|MOUS_TO_LOCAL_MOBILES|MOUS_FROM_LOCAL_MOBILES|MOUS_TO_LOCAL_LANDLINES|MOUS_FROM_LOCAL_LANDLINES|MOUS_TO_INT_NUMBER|MOUS_FROM_INT_NUMBER|     DATA_IN_BNDL|DATA_OUT_BNDL|DATA_USG_PAYG|COMPLAINTS|Years_stayed|PTY_PROFILE_SUB_TYPE|PARTY_GENDER_CD|STATUS|SOCIO_ECONOMIC_SEGMENT|PARTY_NATIONALITY|
+---+------+-----------+------------+------------------+----------

In [9]:
df_spark_col.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- TARGET: string (nullable = true)
 |-- YEAR_JOINED: string (nullable = true)
 |-- CURRENT_YEAR: string (nullable = true)
 |-- BILL_AMOUNT: string (nullable = true)
 |-- PAID_AMOUNT: string (nullable = true)
 |-- PAYMENT_TRANSACTIONS: string (nullable = true)
 |-- PARTY_REV: string (nullable = true)
 |-- PREPAID_LINES: string (nullable = true)
 |-- POSTPAID_LINES: string (nullable = true)
 |-- OTHER_LINES: string (nullable = true)
 |-- LINE_REV: string (nullable = true)
 |-- MOUS_TO_LOCAL_MOBILES: string (nullable = true)
 |-- MOUS_FROM_LOCAL_MOBILES: string (nullable = true)
 |-- MOUS_TO_LOCAL_LANDLINES: string (nullable = true)
 |-- MOUS_FROM_LOCAL_LANDLINES: string (nullable = true)
 |-- MOUS_TO_INT_NUMBER: string (nullable = true)
 |-- MOUS_FROM_INT_NUMBER: string (nullable = true)
 |-- DATA_IN_BNDL: string (nullable = true)
 |-- DATA_OUT_BNDL: string (nullable = true)
 |-- DATA_USG_PAYG: string (nullable = true)
 |-- COMPLAINTS: string (n

Se hace una conversión de datos string a float o integer según la columna, si bien el set de datos ya estaba en términos numéricos, Spark los consideró como cadenas al crear el dataframe inicial

In [10]:
for column in df_spark_col.columns:
  if column not in ["_c0", "TARGET", "YEAR_JOINED", "CURRENT_YEAR", "PAYMENT_TRANSACTIONS", "PREPAID_LINES", "POSTPAID_LINES", "OTHER_LINES", "COMPLAINTS", "PARTY_GENDER_CD", "STATUS", "SOCIO_ECONOMIC_SEGMENT", "PARTY_NATIONALITY"]:
    df_spark_col = df_spark_col.withColumn(column, col(column).cast(FloatType()))
  else:
    df_spark_col = df_spark_col.withColumn(column, col(column).cast(IntegerType()))


df_spark_col.show()

+---+------+-----------+------------+-----------+-----------+--------------------+---------+-------------+--------------+-----------+---------+---------------------+-----------------------+-----------------------+-------------------------+------------------+--------------------+------------+-------------+-------------+----------+------------+--------------------+---------------+------+----------------------+-----------------+
|_c0|TARGET|YEAR_JOINED|CURRENT_YEAR|BILL_AMOUNT|PAID_AMOUNT|PAYMENT_TRANSACTIONS|PARTY_REV|PREPAID_LINES|POSTPAID_LINES|OTHER_LINES| LINE_REV|MOUS_TO_LOCAL_MOBILES|MOUS_FROM_LOCAL_MOBILES|MOUS_TO_LOCAL_LANDLINES|MOUS_FROM_LOCAL_LANDLINES|MOUS_TO_INT_NUMBER|MOUS_FROM_INT_NUMBER|DATA_IN_BNDL|DATA_OUT_BNDL|DATA_USG_PAYG|COMPLAINTS|Years_stayed|PTY_PROFILE_SUB_TYPE|PARTY_GENDER_CD|STATUS|SOCIO_ECONOMIC_SEGMENT|PARTY_NATIONALITY|
+---+------+-----------+------------+-----------+-----------+--------------------+---------+-------------+--------------+-----------+-------

In [11]:
df_spark_col.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- TARGET: integer (nullable = true)
 |-- YEAR_JOINED: integer (nullable = true)
 |-- CURRENT_YEAR: integer (nullable = true)
 |-- BILL_AMOUNT: float (nullable = true)
 |-- PAID_AMOUNT: float (nullable = true)
 |-- PAYMENT_TRANSACTIONS: integer (nullable = true)
 |-- PARTY_REV: float (nullable = true)
 |-- PREPAID_LINES: integer (nullable = true)
 |-- POSTPAID_LINES: integer (nullable = true)
 |-- OTHER_LINES: integer (nullable = true)
 |-- LINE_REV: float (nullable = true)
 |-- MOUS_TO_LOCAL_MOBILES: float (nullable = true)
 |-- MOUS_FROM_LOCAL_MOBILES: float (nullable = true)
 |-- MOUS_TO_LOCAL_LANDLINES: float (nullable = true)
 |-- MOUS_FROM_LOCAL_LANDLINES: float (nullable = true)
 |-- MOUS_TO_INT_NUMBER: float (nullable = true)
 |-- MOUS_FROM_INT_NUMBER: float (nullable = true)
 |-- DATA_IN_BNDL: float (nullable = true)
 |-- DATA_OUT_BNDL: float (nullable = true)
 |-- DATA_USG_PAYG: float (nullable = true)
 |-- COMPLAINTS: integer (nulla

Se procede a convertir el dataframe a una PipelinedRDD con tal de que el modelo de MlLib seleccionado pueda implementarse

In [12]:
df_RDD = df_spark_col.rdd.map(lambda x: LabeledPoint(x[1], x[2:]))
df_RDD.take(10)

[LabeledPoint(0.0, [1994.0,2019.0,931.2089233398438,812.1749877929688,1.0,5968.7001953125,2.0,6.0,2.0,945.0399780273438,1004.0700073242188,35.849998474121094,34.01499938964844,72.07499694824219,141.83999633789062,56.1150016784668,11944.0791015625,0.0,0.0,0.0,25.0,2.0,1.0,0.0,0.0,179.0]),
 LabeledPoint(0.0, [1994.0,2019.0,431.0826110839844,486.5,1.0,6245.1416015625,6.0,4.0,2.0,493.81500244140625,159.0500030517578,10.595000267028809,7.715000152587891,11.75,5.110000133514404,0.0,9903.158203125,0.0,0.0,0.0,25.0,1.0,1.0,0.0,0.0,179.0]),
 LabeledPoint(0.0, [1994.0,2019.0,50.61964416503906,52.814998626708984,1.0,1666.48828125,2.0,3.0,1.0,50.29999923706055,0.0,0.0,0.0,0.0,0.0,0.0,0.1025390625,0.0,0.0,0.0,25.0,2.0,1.0,0.0,0.0,179.0]),
 LabeledPoint(0.0, [1994.0,2019.0,399.71002197265625,422.2349853515625,1.0,2522.00830078125,3.0,4.0,3.0,406.586669921875,288.80499267578125,158.5,2.6700000762939453,15.96500015258789,0.0,0.0,3600.322265625,0.0,0.0,0.0,25.0,1.0,1.0,0.0,0.0,179.0]),
 LabeledPoint(0.

División del PipelinedRDD en test y train

In [13]:
(train, test) = df_RDD.randomSplit([0.7,0.3])


Implementación de regresión logística con LBFGS en su configuración estándar entrenando con el set de datos específico para el entranamiento

In [14]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel

model = LogisticRegressionWithLBFGS.train(train)

Se obtienen las predicciones de churn o no churn de los clientes basándose en el set de pruebas así como la obtención del error del entrenamiento, siendo de un 5.23%

In [15]:
labelsAndPreds = test.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda lp: lp[0] != lp[1]).count() / float(test.count())
print("Training Error = " + str(trainErr))

Training Error = 0.05235111091634165


Intercepto del modelo

In [16]:
print("Logistic Regression Intercept = ", model.intercept)

Logistic Regression Intercept =  0.0


In [17]:
y_pred = model.predict(train)
y_pred

PythonRDD[182] at RDD at PythonRDD.scala:53

100 predicciones para mostrar el comportamiento de los resultados, siendo en su mayoría los clientes con 0 de valor (no churn)

In [18]:
labelsAndPreds.take(100)

[(0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (1.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (1.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),

Se obtuvieron 18033 clientes con posible churn y 324252 con no churn

In [19]:
churn = []
no_churn = []
pred = labelsAndPreds.collect()
for row in pred:
  if row[0] == 0.0:
    no_churn.append(row)
  else:
    churn.append(row)

print("Clients with churn: ", len(churn))
print("Clients with no churn: ", len(no_churn))

Clients with churn:  18033
Clients with no churn:  324252


A su vez, la regresión logística cuenta con un 87.7% de efectividad

In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

dataset = spark_session.createDataFrame(labelsAndPreds, ["raw", "label"])
evaluator = BinaryClassificationEvaluator()
evaluator.setRawPredictionCol("raw")
evaluator.evaluate(dataset)


0.8770886356326395

### **Gráficas de Tableau**



Gráfica 1: Llamadas a teléfono fijo o celular según segmento socioeconómico
<br>
<img src = "https://drive.google.com/uc?id=1dFRdtadg73VUilIpipQ-1nAQVtWjbkgG" alt = "Gráfica 1" />

Gráfica 2: Nacionalidad vs Bill Amount

<img src = "https://drive.google.com/uc?id=14t7A3ado8myF5SX-dtAJz6h9TlFRCftO" alt = "Gráfica 2" />

Gráfica 3: Quejas según el año en el que se unió el cliente

<img src = "https://drive.google.com/uc?id=1MtykbLpe1XsxA5j2BS8hvbqBVnXD4Mst" alt = "Gráfica 3" />

Gráfica 4: Años en la empresa de cada cliente por lineas contratadas

<img src = "https://drive.google.com/uc?id=1Wi2SQlYxWEtMFsrB7ihJMCfOYiuzRjSp" alt = "Gráfica 4" />
